In [1]:
import pandas as pd

base = pd.read_csv('dataset.csv')

In [2]:
import numpy as np

previsores = base.iloc[:1400000,0:20].values
previsores = np.reshape(previsores, (2000, 700, 20))
print(previsores.shape)

classes = base.iloc[:1400000, 20:28].values
classes = np.reshape(classes, (2000, 700, 8))
print(classes.shape)

(2000, 700, 20)
(2000, 700, 8)


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM, Bidirectional, Masking

In [6]:
def criarRede():
    model = Sequential()
  
    #model.add(Masking(mask_value = 0, input_shape = (700, 20)))
  
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True), input_shape = (700, 20)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))

    model.add(Bidirectional(CuDNNLSTM(100, return_sequences = True)))
    model.add(Dropout(0.3))
    
    model.add(Dense(8, activation = 'softmax'))
  
    model.compile(optimizer = 'adam', metrics = ['acc'], loss='categorical_crossentropy')
  
    return model

In [7]:
accuq8 = []
precisionsq8 = []
recallsq8 = []
f1q8 = []

In [8]:
from sklearn.metrics import balanced_accuracy_score, classification_report, precision_score, recall_score, f1_score, confusion_matrix

def train_and_evaluate_model(model, x_train, y_train, x_test, y_test):
    model.fit(x_train, y_train, epochs = 100, verbose = 1, batch_size = 32)
  
    predicted = model.predict(x_test)
  
    y_teste = []
    predict = []
  
    predicted = np.reshape(predicted, (predicted.shape[0] * predicted.shape[1], 8))
    y_test = np.reshape(y_test, (y_test.shape[0] * y_test.shape[1], 8))
    x_test = np.reshape(x_test, (x_test.shape[0] * x_test.shape[1], 20))

    for i in range(len(x_test)):
        cont = 0
        for j in range(len(x_test[i])):
            cont += x_test[i][j]
        if cont != 0:
            y_teste.append(y_test[i])
            predict.append(predicted[i])
    
    y_teste = np.asarray(y_teste)
    predict = np.asarray(predict)

    predicted = predict
    y_test = y_teste

    predicted = np.argmax(predicted, axis=1)
    y_test = np.argmax(y_test, axis=1)

    accuq8.append(balanced_accuracy_score(y_test, predicted)) 
    precisionsq8.append(precision_score(y_test, predicted, average = 'weighted'))
    recallsq8.append(recall_score(y_test, predicted, average = 'weighted'))
    f1q8.append(f1_score(y_test, predicted, average = 'weighted'))
    
    print(classification_report(y_test, predicted))
    print('Acurácia')
    print(balanced_accuracy_score(y_test, predicted))
    print('Precisao')
    print(precision_score(y_test, predicted, average = 'weighted'))
    print('Recall')
    print(recall_score(y_test, predicted, average = 'weighted'))
    print('F1')
    print(f1_score(y_test, predicted, average = 'weighted'))
    print(confusion_matrix(y_test, predicted))

In [9]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = True)

for train_index, test_index in kf.split(previsores):
    model = None
    model = criarRede()
    print("TRAIN:", train_index, "TEST:", test_index)
    train_and_evaluate_model(model, previsores[train_index], classes[train_index],
                           previsores[test_index], classes[test_index])

W0818 22:42:08.317436  9768 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 22:42:08.322394  9768 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0818 22:42:08.323392  9768 deprecation.py:506] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\init_ops.py

TRAIN: [   0    1    2 ... 1997 1998 1999] TEST: [   5    8    9   13   16   18   21   22   30   36   38   39   54   58
   59   61   62   63   68   69   70   77   82   92   96   98  106  112
  129  136  143  145  150  158  162  170  176  177  178  185  187  188
  191  192  198  200  212  213  223  228  231  237  242  247  249  250
  251  253  259  265  267  268  273  276  286  292  298  300  304  312
  329  351  354  355  364  368  369  376  379  380  389  394  404  406
  407  410  413  423  429  431  432  433  435  437  441  446  454  457
  468  469  472  498  501  506  508  510  526  531  538  540  544  550
  553  555  564  566  572  581  584  599  600  602  603  615  617  618
  623  624  626  631  635  637  655  657  662  664  675  680  681  685
  694  699  702  704  708  720  722  728  737  746  747  749  754  762
  768  772  775  778  780  787  788  792  793  804  805  808  811  813
  816  827  829  830  839  842  848  854  858  860  866  867  875  878
  879  891  892  897  902  9

W0818 22:42:11.176790  9768 deprecation.py:323] From c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
1600/1600 [==============================] - 31s 19ms/sample - loss: 0.6419 - acc: 0.1278
Epoch 2/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6304 - acc: 0.1224
Epoch 3/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6274 - acc: 0.1250
Epoch 4/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6267 - acc: 0.1250
Epoch 5/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6255 - acc: 0.1253
Epoch 6/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6250 - acc: 0.1254
Epoch 7/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6253 - acc: 0.1253
Epoch 8/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1322
Epoch 9/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.5961 - acc: 0.1430
Epoch 10/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2865 - acc: 0.2612
Epoch 81/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2833 - acc: 0.2622
Epoch 82/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2822 - acc: 0.2627
Epoch 83/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2796 - acc: 0.2635
Epoch 84/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2754 - acc: 0.2648
Epoch 85/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2749 - acc: 0.2653
Epoch 86/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2734 - acc: 0.2655
Epoch 87/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2722 - acc: 0.2660
Epoch 88/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.2709 - acc: 0.2667
Epoch 89/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.26

...

Epoch 1/100
1600/1600 [==============================] - 29s 18ms/sample - loss: 0.6346 - acc: 0.1275
Epoch 2/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6250 - acc: 0.1212
Epoch 3/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6234 - acc: 0.1230
Epoch 4/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6217 - acc: 0.1236
Epoch 5/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6256 - acc: 0.1446
Epoch 6/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6231 - acc: 0.1230
Epoch 7/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6220 - acc: 0.1232
Epoch 8/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6216 - acc: 0.1232
Epoch 9/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6215 - acc: 0.1234
Epoch 10/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1235
Epoch 81/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1235
Epoch 82/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1235
Epoch 83/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6193 - acc: 0.1235
Epoch 84/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1235
Epoch 85/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6189 - acc: 0.1235
Epoch 86/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6195 - acc: 0.1235
Epoch 87/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6192 - acc: 0.1235
Epoch 88/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.6191 - acc: 0.1235
Epoch 89/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.61

c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\gabriel\appdata\local\programs\python\python36\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1354
           1       0.32      0.02      0.05     21179
           2       0.00      0.00      0.00      4064
           3       0.34      0.99      0.50     34368
           4       0.00      0.00      0.00       603
           5       0.70      0.04      0.08     21177
           6       0.00      0.00      0.00      8989
           7       0.00      0.00      0.00     11698

    accuracy                           0.34    103432
   macro avg       0.17      0.13      0.08    103432
weighted avg       0.32      0.34      0.19    103432

Acurácia
0.13117469479562668
Precisao
0.32080741850927075
Recall
0.34054257869904864
F1
0.19132318609369187
[[    0    23     0  1313     0    18     0     0]
 [    0   513     0 20520     0   146     0     0]
 [    0    53     0  3993     0    18     0     0]
 [    0   397     0 33870     0   101     0     0]
 [    0     0     0   603     0     0

1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4524 - acc: 0.2074
Epoch 50/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4499 - acc: 0.2081
Epoch 51/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4439 - acc: 0.2107
Epoch 52/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4376 - acc: 0.2127
Epoch 53/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4324 - acc: 0.2143
Epoch 54/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4284 - acc: 0.2160
Epoch 55/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4249 - acc: 0.2171
Epoch 56/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4200 - acc: 0.2185
Epoch 57/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.4166 - acc: 0.2196
Epoch 58/100
1600/1600 [==============================] - 27s 17ms/sample - loss: 0.41

In [11]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_40 (Bidirectio (None, 700, 200)          97600     
_________________________________________________________________
dropout_40 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_41 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_41 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_42 (Bidirectio (None, 700, 200)          241600    
_________________________________________________________________
dropout_42 (Dropout)         (None, 700, 200)          0         
_________________________________________________________________
bidirectional_43 (Bidirectio (None, 700, 200)         

In [12]:
print('Acurácias total')
print(accuq8)
a = np.array(accuq8)
print(a.mean())
print('Precision total')
print(precisionsq8)
p = np.array(precisionsq8)
print(p.mean())
print('Recalls total')
print(recallsq8)
r = np.array(recallsq8)
print(r.mean())
print('F1 total')
print(f1q8)
f = np.array(f1q8)
print(f.mean())

Acurácias total
[0.40017190541695397, 0.13150230690085424, 0.38899528619781787, 0.13117469479562668, 0.37063193247336323]
0.2844952251569232
Precision total
[0.5784424192514481, 0.3057907368040687, 0.5535203867730304, 0.32080741850927075, 0.563163871772488]
0.46434496662206115
Recalls total
[0.5897071537169716, 0.32891466445733225, 0.5559197600084342, 0.34054257869904864, 0.5694436209111572]
0.4769055555585887
F1 total
[0.5728260266120115, 0.18071996959574058, 0.5462962995575783, 0.19132318609369187, 0.5456447036823472]
0.4073620371082739
